## Settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import random
import os
import sys
import logging
import torch

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
project_root = os.path.abspath('')
if project_root not in sys.path:
    sys.path.append(project_root)
project_root

'/Users/meina/Github/meina-t/matching_with_dl'

In [4]:
from logging import getLogger
logger = getLogger(__name__)
logger.setLevel(logging.INFO)

sv 
https://arxiv.org/pdf/2107.03427


## Train

In [5]:
from model.model import MatchingNet, train_model
from model.params import HParams
from model.data import Data
from model.loss.strategy_proofness import compute_spv
from model.loss.stability import compute_sv
from model.loss.efficiency import generate_stable_matchings, filter_efficient_stable_matchings, compute_ev
from model.utils import is_pareto_dominates

In [6]:
cfg = HParams(
    num_agents=3,
    num_hidden_nodes=64,
    batch_size=128,
    epochs=2000,
    corr = 0,
    device='mps',
    lr=0.01,
    )

In [7]:
model = MatchingNet(cfg)

In [8]:
data = Data(cfg)

In [9]:
train_model(cfg, model, data)

Training started.
Epoch: 10
Total Loss: 4.965608596801758
SPV: 0.0035193879157304764
SV: 0.658410906791687
Objective Loss(ev): 0.3297705352306366
Paramiters: lambda_spv = 1.1498084566555917, lambda_sv = 7.693232774734497, rho = 1
---------------------------
Epoch: 20
Total Loss: 9.773368835449219
SPV: 0.0036173106636852026
SV: 0.6685671806335449
Objective Loss(ev): 0.6563866138458252
Paramiters: lambda_spv = 1.1780450765509158, lambda_sv = 14.298810422420502, rho = 1
---------------------------
Epoch: 30
Total Loss: 14.03762149810791
SPV: 0.034768588840961456
SV: 0.6532100439071655
Objective Loss(ev): 0.8628828525543213
Paramiters: lambda_spv = 1.335722693009302, lambda_sv = 20.753185093402863, rho = 1
---------------------------
Epoch: 40
Total Loss: 14.388965606689453
SPV: 0.46244657039642334
SV: 0.47323372960090637
Objective Loss(ev): 0.4681311845779419
Paramiters: lambda_spv = 4.01011608238332, lambda_sv = 26.42283606529236, rho = 1
---------------------------
Epoch: 50
Total Loss:

In [10]:
torch.save(model.state_dict(), 'model_0120.pth')

In [10]:
model

MatchingNet(
  (layers): Sequential(
    (0): Linear(in_features=18, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=64, out_features=9, bias=True)
  )
)

In [11]:
device = 'mps'

p = torch.tensor([[[1, 0, 0.0], [0., 1, 0], [0., 0, 1]]],  dtype=torch.float32).to(device)
q = torch.tensor([[[0, 0.0, 1], [0.0, 1, 0.0], [0.3, 0.3, 1]]],  dtype=torch.float32).to(device)

In [12]:
model(p, q)

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e+00, 0.0000e+00],
         [9.9657e-06, 0.0000e+00, 9.9999e-01]]], device='mps:0',
       grad_fn=<DivBackward0>)

In [23]:
r = model(p, q)

In [ ]:
compute_sv(cfg, r, p, q)

tensor([[0.4611, 0.0000, 0.0000],
        [0.0000, 0.4510, 0.0000],
        [0.0000, 0.0000, 0.4547]], device='mps:0', grad_fn=<MeanBackward1>)

In [36]:
import itertools

In [37]:
p_list = list(itertools.permutations([3, 2, 1]))
q_list = p_list + [(1,2,2), (2,1,2), (2,2,1),(1,1,2), (1,2,1), (2,1,1), (1,1,1)]

In [38]:
pairs_1_1 = []
pairs_1_2 = []
pairs_1_3 = []
pairs_2_1 = []
pairs_2_2 = []
pairs_2_3 = []
pairs_3_1 = []
pairs_3_2 = []
pairs_3_3 = []

p_set_1 = []
p_set_2 = []
p_set_3 = []
q_set_1 = []
q_set_2 = []
q_set_3 = []

for i in range(6):
    p_1 = p_list[i]
    p_set_1.append([p_1 for _ in range(3)])
for i in range(13):
    q_1 = q_list[i]
    q_set_1.append([q_1 for _ in range(3)])

# p_listから重複なく2つ選び、p_2を作成
for i in range(5):
    for j in range(i+1, 6):
        p_set_2.append([p_list[i], p_list[j], p_list[j]])

for i in range(12):
    for j in range(i+1, 13):
        q_set_2.append([q_list[i], q_list[j], q_list[j]])

for i in range(4):
    for j in range(i+1, 5):
        for k in range(j+1, 6):
            p_set_3.append([p_list[i], p_list[j], p_list[k]])

for i in range(11):
    for j in range(i+1, 12):
        for k in range(j+1, 13):
            q_set_3.append([q_list[i], q_list[j], q_list[k]])



for i in p_set_1:
    for j in q_set_1:
        pairs_1_1.append((i, j))

for i in p_set_1:
    for j in q_set_2:
        pairs_1_2.append((i, j))

for i in p_set_1:
    for j in q_set_3:
        pairs_1_3.append((i, j))

for i in p_set_2:
    for j in q_set_1:
        pairs_2_1.append((i, j))

for i in p_set_2:
    for j in q_set_2:
        pairs_2_2.append((i, j))

for i in p_set_2:
    for j in q_set_3:
        pairs_2_3.append((i, j))

for i in p_set_3:
    for j in q_set_1:
        pairs_3_1.append((i, j))

for i in p_set_3:
    for j in q_set_2:
        pairs_3_2.append((i, j))

for i in p_set_3:
    for j in q_set_3:
        pairs_3_3.append((i, j))

In [40]:
df_1_1 = pd.DataFrame(pairs_1_1, columns=['p', 'q'])
df_1_2 = pd.DataFrame(pairs_1_2, columns=['p', 'q'])
df_1_3 = pd.DataFrame(pairs_1_3, columns=['p', 'q'])
df_2_1 = pd.DataFrame(pairs_2_1, columns=['p', 'q'])
df_2_2 = pd.DataFrame(pairs_2_2, columns=['p', 'q'])
df_2_3 = pd.DataFrame(pairs_2_3, columns=['p', 'q'])
df_3_1 = pd.DataFrame(pairs_3_1, columns=['p', 'q'])
df_3_2 = pd.DataFrame(pairs_3_2, columns=['p', 'q'])
df_3_3 = pd.DataFrame(pairs_3_3, columns=['p', 'q'])

In [42]:
from old.utils import apply_features, convert_to_float

In [47]:
df_1_1 = apply_features(cfg, model, df_1_1, data)

Processing rows:   0%|          | 0/78 [00:00<?, ?it/s]


RuntimeError: a Tensor with 9 elements cannot be converted to Scalar